In [1]:
import os
from brtdevkit.util.aws import s3

Internal AWS credentials will be removed from future brt-devkit versions.


In [2]:
s3_client = s3.S3()
BRT_DEVKIT_WRITE_S3_BUCKET = 'mesa-states'
MODEL_TRAINING_S3_KEY_PREFIX = f"{os.environ.get('BRT_ENV', 'prod')}/jupiter/model_training"
PACK_PERCEPTION_S3_KEY_PREFIX = f"{os.environ.get('BRT_ENV', 'prod')}/jupiter/pack_perception"
bucket_name=BRT_DEVKIT_WRITE_S3_BUCKET

In [ ]:
# upload file
# key = f"{MODEL_TRAINING_S3_KEY_PREFIX}/Jupiter_train_v4_53_missing_human_relabeled.tar"
key = f"{PACK_PERCEPTION_S3_KEY_PREFIX}/Jupiter_train_v4_53_heavy_dust_relabeled.tar"
file_path = "/home/bluerivertech/li.yu/data/Jupiter_train_v4_53_heavy_dust_relabeled/Jupiter_train_v4_53_heavy_dust_relabeled.tar"
s3_client.upload_file(
    bucket_name=bucket_name, key=key, file_path=file_path)
# Jupiter_2022_fn_bag1.tar
# Jupiter_2022Jan15Feb15_rock_stops.tar
# Jupiter_train_v4_53_missing_human_relabeled.tar
# Jupiter_train_v4_53_heavy_dust_relabeled.tar

In [4]:
# download file
# run_id = '45_3_6class_ml_pp_seg_lite_cloud_v10_rotate_fix'
# name = 'driveable_terrain_model_val_bestmodel.pth'
# key = f"{MODEL_TRAINING_S3_KEY_PREFIX}/{run_id}/{name}"

key = f"{PACK_PERCEPTION_S3_KEY_PREFIX}/Jupiter_train_v4_53_heavy_dust_relabeled.tar"

target_path = '/data/jupiter/li.yu/data/Jupiter_train_v4_53/Jupiter_train_v4_53_heavy_dust_relabeled/Jupiter_train_v4_53_heavy_dust_relabeled.tar'
s3_client.download_file(bucket_name, key, target_path)

True

In [2]:
import torch

In [3]:
pth = '/data/jupiter/li.yu/exps/driveable_terrain_model/openimages_v7_nextvit_base_512x640_ft_0211/checkpoint.pth'
checkpoint = torch.load(pth, map_location=lambda storage, loc: storage)
type(checkpoint)

dict

In [6]:
keys = checkpoint['model'].keys()
keys

odict_keys(['stem.0.conv.weight', 'stem.0.norm.weight', 'stem.0.norm.bias', 'stem.0.norm.running_mean', 'stem.0.norm.running_var', 'stem.0.norm.num_batches_tracked', 'stem.1.conv.weight', 'stem.1.norm.weight', 'stem.1.norm.bias', 'stem.1.norm.running_mean', 'stem.1.norm.running_var', 'stem.1.norm.num_batches_tracked', 'stem.2.conv.weight', 'stem.2.norm.weight', 'stem.2.norm.bias', 'stem.2.norm.running_mean', 'stem.2.norm.running_var', 'stem.2.norm.num_batches_tracked', 'stem.3.conv.weight', 'stem.3.norm.weight', 'stem.3.norm.bias', 'stem.3.norm.running_mean', 'stem.3.norm.running_var', 'stem.3.norm.num_batches_tracked', 'features.0.patch_embed.conv.weight', 'features.0.patch_embed.norm.weight', 'features.0.patch_embed.norm.bias', 'features.0.patch_embed.norm.running_mean', 'features.0.patch_embed.norm.running_var', 'features.0.patch_embed.norm.num_batches_tracked', 'features.0.mhca.group_conv3x3.weight', 'features.0.mhca.norm.weight', 'features.0.mhca.norm.bias', 'features.0.mhca.norm.